<a href="https://colab.research.google.com/github/pragya183rashmi/checkin/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [2]:
pip install gradio transformers gtts

In [3]:
import gradio as gr
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from gtts import gTTS
import uuid
import os

# Load BlenderBot model
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

# Emotion-based custom replies
def emotion_response(text):
    lower = text.lower()
    if "happy" in lower or "excited" in lower:
        return "That's wonderful to hear! 🌞 What's been bringing you joy lately?"
    elif "tired" in lower or "exhausted" in lower:
        return "You’ve done so much. 🛌 Maybe it’s time to rest and be kind to yourself."
    elif "angry" in lower or "frustrated" in lower:
        return "It’s okay to feel that way. 😤 Want to talk about what triggered it?"
    elif "anxious" in lower or "nervous" in lower:
        return "Breathe with me for a moment. 🌬️ You’re safe now. What’s worrying you most?"
    elif "sad" in lower or "down" in lower:
        return "I'm here for you. 💙 Want to share what's been making you feel this way?"
    return None

# AI response function
def ai_response(user_input, history=[]):
    emotion_reply = emotion_response(user_input)
    if emotion_reply:
        bot_response = emotion_reply
    else:
        inputs = tokenizer(user_input, return_tensors="pt")
        reply_ids = model.generate(**inputs)
        bot_response = tokenizer.decode(reply_ids[0], skip_special_tokens=True)

    history.append((user_input, bot_response))

    tts = gTTS(bot_response)
    audio_path = f"/tmp/audio_{uuid.uuid4()}.mp3"
    tts.save(audio_path)

    return history, audio_path

# Gradio UI setup
with gr.Blocks(theme=gr.themes.Soft(), css="""
body::before {
    content: "";
    position: fixed;
    top: 0; left: 0;
    width: 100%; height: 100%;
    background-image: url('https://images.unsplash.com/photo-1506744038136-46273834b3fb?auto=format&fit=crop&w=1920&q=80');
    background-size: cover;
    background-repeat: no-repeat;
    background-attachment: fixed;
    background-position: center;
    opacity: 0.6;
    z-index: -1;
}
.gr-box, .gr-block, .gr-textbox, .gr-button, .gr-audio, .gr-chatbot {
    background-color: rgba(255, 255, 255, 0.85) !important;
    border-radius: 12px;
    padding: 10px;
}
""") as demo:
    gr.Markdown("""
    <div style="text-align: center;">
        <h1 style="color: #7F5AF0;">🧠 Emotional Support Chat</h1>
        <p style="font-size: 16px; color: #333;">Talk to me about anything. I'm here to listen, support, and respond with empathy. 💬</p>
    </div>
    """)

    chatbot = gr.Chatbot(label="Your Conversation")
    msg = gr.Textbox(placeholder="How are you feeling today?", label="You", lines=2)
    audio_output = gr.Audio(label="AI Voice Response", interactive=False)
    state = gr.State([])

    def handle_submit(user_msg, chat_state):
        chat_state, audio = ai_response(user_msg, chat_state)
        return "", chat_state, chat_state, audio

    with gr.Row():
        send_btn = gr.Button("💬 Send", variant="primary")

    send_btn.click(handle_submit, inputs=[msg, state], outputs=[msg, state, chatbot, audio_output])
    msg.submit(handle_submit, inputs=[msg, state], outputs=[msg, state, chatbot, audio_output])

    gr.Markdown("""
    <p style="text-align: center; font-size: 14px; color: #444;">Made with ❤️ to help you feel heard.</p>
    """)

# Launch the app
demo.launch()
background: url('https://images.unsplash.com/photo-1506744038136-46273834b3fb?auto=format&fit=crop&w=1920&q=80')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

<ipython-input-3-b2520298bd37>:73: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Your Conversation")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4ac8afd4d890ba70a9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


NameError: name 'url' is not defined